# Try to predict new prices using the previous manual process

In [165]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import joblib

In [166]:
test_data = pd.read_csv('test.csv')
test_data.drop('Id', axis=1, inplace=True)
test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Feature engineering

We need to apply to the test data the same process that we applied to the train data

### Missing values treatment
Replace missing values with the string "missing" in those variables with a lot of missing data.

Alternatively, we will replace missing data with the most frequent category in those variables that contain fewer observations without values.

In [167]:
cat_vars = [var for var in test_data.columns if test_data[var].dtype == 'O']

cat_vars = cat_vars + ['MSSubClass']

test_data[cat_vars] = test_data[cat_vars].astype('O')

len(cat_vars)

44

In [168]:
# make a list of the categorical variables that contain missing values

cat_vars_with_na = [
    var for var in cat_vars
    if test_data[var].isnull().sum() > 0
]

test_data[cat_vars_with_na].isnull().mean().sort_values(ascending=False)

PoolQC          0.997944
MiscFeature     0.965045
Alley           0.926662
Fence           0.801234
FireplaceQu     0.500343
GarageCond      0.053461
GarageQual      0.053461
GarageFinish    0.053461
GarageType      0.052090
BsmtCond        0.030843
BsmtQual        0.030158
BsmtExposure    0.030158
BsmtFinType1    0.028787
BsmtFinType2    0.028787
MasVnrType      0.010966
MSZoning        0.002742
Functional      0.001371
Utilities       0.001371
KitchenQual     0.000685
Exterior2nd     0.000685
Exterior1st     0.000685
SaleType        0.000685
dtype: float64

In [169]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cat_vars_with_na if test_data[var].isnull().mean() > 0.1
]

# variables to impute with the most frequent category
with_frequent_category = [
    var for var in cat_vars_with_na if test_data[var].isnull().mean() < 0.1
]

with_string_missing

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [170]:
# replace missing values with new label: "Missing"

test_data[with_string_missing] = test_data[with_string_missing].fillna('Missing')

In [171]:
frequent_category_dict = {}

for var in with_frequent_category:
    mode = test_data[var].mode()[0]
    frequent_category_dict[var] = mode

test_data.fillna(value=frequent_category_dict, inplace=True)

In [172]:
# check that we have no missing information in the engineered variables

test_data[list(frequent_category_dict.keys()) + with_string_missing].isnull().sum()

MSZoning        0
Utilities       0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
KitchenQual     0
Functional      0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
SaleType        0
Alley           0
FireplaceQu     0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [173]:
test_data[cat_vars].isna().sum()

MSZoning         0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
PoolQC           0
Fence            0
MiscFeature      0
SaleType         0
SaleCondition    0
MSSubClass       0
dtype: int64

### Missing values in numerical variables
To engineer missing values in numerical variables, we will:
- add a binary missing indicator variable
- and then replace the missing values in the original variable with the mean

In [174]:
num_vars = [
    var for var in test_data.columns if var not in cat_vars and var != 'SalePrice'
]

len(num_vars)

35

In [175]:
# make a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in num_vars
    if test_data[var].isnull().sum() > 0
]

test_data[vars_with_na].isnull().mean()

LotFrontage     0.155586
MasVnrArea      0.010281
BsmtFinSF1      0.000685
BsmtFinSF2      0.000685
BsmtUnfSF       0.000685
TotalBsmtSF     0.000685
BsmtFullBath    0.001371
BsmtHalfBath    0.001371
GarageYrBlt     0.053461
GarageCars      0.000685
GarageArea      0.000685
dtype: float64

In [176]:
# replace missing values as we described above

for var in vars_with_na:
    mean_val = test_data[var].mean()
    print(var, mean_val)

    # add binary missing indicator
    test_data[var + '_na'] = np.where(test_data[var].isnull(), 1, 0)

    # replace missing values by the mean
    test_data[var].fillna(mean_val, inplace=True)

# check that we have no more missing values in the engineered variables
test_data[vars_with_na].isnull().sum()

LotFrontage 68.58035714285714
MasVnrArea 100.70914127423822
BsmtFinSF1 439.2037037037037
BsmtFinSF2 52.61934156378601
BsmtUnfSF 554.2949245541838
TotalBsmtSF 1046.1179698216736
BsmtFullBath 0.4344543582704187
BsmtHalfBath 0.06520247083047358
GarageYrBlt 1977.7212165097756
GarageCars 1.7661179698216736
GarageArea 472.76886145404666


LotFrontage     0
MasVnrArea      0
BsmtFinSF1      0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
GarageCars      0
GarageArea      0
dtype: int64

In [177]:
missing_indicator_columns = [
    col + '_na' for col in vars_with_na
]

test_data[missing_indicator_columns]

,LotFrontage_na,MasVnrArea_na,BsmtFinSF1_na,BsmtFinSF2_na,BsmtUnfSF_na,TotalBsmtSF_na,BsmtFullBath_na,BsmtHalfBath_na,GarageYrBlt_na,GarageCars_na,GarageArea_na
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,0,0,0,0,0,0,1,0,0
1455,0,0,0,0,0,0,0,0,0,0,0
1456,0,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,0,0,0,1,0,0


### Temporal variables

In [178]:
def elapsed_years(df, var):
    # capture difference between the year variable
    # and the year in which the house was sold
    df[var] = df['YrSold'] - df[var]
    return df

In [179]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    test_data = elapsed_years(test_data, var)

In [180]:
test_data.drop('YrSold', axis=1, inplace=True)

### Numerical variable transformation

In [181]:
for var in ['LotFrontage', '1stFlrSF', 'GrLivArea']:
    test_data[var] = np.log(test_data[var])

In [182]:
# check that test set does not contain null values in the engineered variables
[var for var in ['LotFrontage', '1stFlrSF', 'GrLivArea'] if test_data[var].isnull().sum() > 0]

[]

### Yeo-Johnson transformation
We will apply the Yeo-Johnson transformation to LotArea

In [183]:
# we apply the yeo-johnson transformation with the same parameter
# that we used during the model training
param = -12.55283001172003
test_data['LotArea'] = stats.yeojohnson(test_data['LotArea'], lmbda=param)

In [184]:
# check absence of na in the data
[var for var in test_data.columns if test_data[var].isnull().sum() > 0]

[]

### Binarize skewed variables

In [185]:
skewed = [
    'BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'MiscVal'
]

for var in skewed:
    # map the variable values into 0 and 1
    test_data[var] = np.where(test_data[var]==0, 0, 1)

### Categorical variables

In [186]:
# re-map strings to numbers, which determine quality

qual_mappings = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
             'HeatingQC', 'KitchenQual', 'FireplaceQu',
             'GarageQual', 'GarageCond',
            ]

for var in qual_vars:
    test_data[var] = test_data[var].map(qual_mappings)

In [187]:
exposure_mappings = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
var = 'BsmtExposure'

test_data[var] = test_data[var].map(exposure_mappings)

In [188]:
finish_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
finish_vars = ['BsmtFinType1', 'BsmtFinType2']

for var in finish_vars:
    test_data[var] = test_data[var].map(finish_mappings)

In [189]:
garage_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
var = 'GarageFinish'

test_data[var] = test_data[var].map(garage_mappings)

In [190]:
fence_mappings = {'Missing': 0, 'NA': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}
var = 'Fence'

test_data[var] = test_data[var].map(fence_mappings)

In [191]:
# check absence of na in the data
[var for var in test_data.columns if test_data[var].isnull().sum() > 0]

[]

### Removing Rare Labels

In [192]:
frequent_labels_dict = {
    'MSZoning': ['FV', 'RH', 'RL', 'RM'],
    'Street': ['Pave'],
    'Alley': ['Grvl', 'Missing', 'Pave'],
    'LotShape': ['IR1', 'IR2', 'Reg'],
    'LandContour': ['Bnk', 'HLS', 'Low', 'Lvl'],
    'Utilities': ['AllPub'],
    'LotConfig': ['Corner', 'CulDSac', 'FR2', 'Inside'],
    'LandSlope': ['Gtl', 'Mod'],
    'Neighborhood': ['Blmngtn', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor',
       'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NWAmes',
       'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW',
       'Somerst', 'StoneBr', 'Timber'],
    'Condition1': ['Artery', 'Feedr', 'Norm', 'PosN', 'RRAn'],
    'Condition2': ['Norm'],
    'BldgType': ['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'],
    'HouseStyle': ['1.5Fin', '1Story', '2Story', 'SFoyer', 'SLvl'],
    'RoofStyle': ['Gable', 'Hip'],
    'RoofMatl': ['CompShg'],
    'Exterior1st': ['AsbShng', 'BrkFace', 'CemntBd', 'HdBoard', 'MetalSd', 'Plywood',
       'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'],
    'Exterior2nd': ['AsbShng', 'BrkFace', 'CmentBd', 'HdBoard', 'MetalSd', 'Plywood',
       'Stucco', 'VinylSd', 'Wd Sdng', 'Wd Shng'],
    'MasVnrType': ['BrkFace', 'None', 'Stone'],
    'Foundation': ['BrkTil', 'CBlock', 'PConc', 'Slab'],
    'Heating': ['GasA', 'GasW'],
    'CentralAir': ['N', 'Y'],
    'Electrical': ['FuseA', 'FuseF', 'SBrkr'],
    'Functional': ['Min1', 'Min2', 'Mod', 'Typ'],
    'GarageType': ['Attchd', 'Basment', 'BuiltIn', 'Detchd'],
    'PavedDrive': ['N', 'P', 'Y'],
    'PoolQC': ['Missing'],
    'MiscFeature': ['Missing', 'Shed'],
    'SaleType': ['COD', 'New', 'WD'],
    'SaleCondition': ['Abnorml', 'Family', 'Normal', 'Partial'],
    'MSSubClass': [20, 30, 50, 60, 70, 75, 80, 85, 90, 120, 160, 190]
}

for key, val in frequent_labels_dict.items():
    test_data[key] = np.where(test_data[key].isin(val), test_data[key], 'Rare')

### Encoding of categorical variables

In [193]:
encoding_dict = {
    'MSZoning': {'Rare': 0, 'RM': 1, 'RH': 2, 'RL': 3, 'FV': 4},
    'Street': {'Rare': 0, 'Pave': 1},
    'Alley': {'Grvl': 0, 'Pave': 1, 'Missing': 2},
    'LotShape': {'Reg': 0, 'IR1': 1, 'Rare': 2, 'IR2': 3},
    'LandContour': {'Bnk': 0, 'Lvl': 1, 'Low': 2, 'HLS': 3},
    'Utilities': {'Rare': 0, 'AllPub': 1},
    'LotConfig': {'Inside': 0, 'FR2': 1, 'Corner': 2, 'Rare': 3, 'CulDSac': 4},
    'LandSlope': {'Gtl': 0, 'Mod': 1, 'Rare': 2},
    'Neighborhood': {'IDOTRR': 0, 'MeadowV': 1, 'BrDale': 2, 'Edwards': 3, 'BrkSide': 4, 'OldTown': 5, 'Sawyer': 6, 'SWISU': 7, 'NAmes': 8, 'Mitchel': 9, 'SawyerW': 10, 'Rare': 11, 'NWAmes': 12, 'Gilbert': 13, 'Blmngtn': 14, 'CollgCr': 15, 'Crawfor': 16, 'ClearCr': 17, 'Somerst': 18, 'Timber': 19, 'StoneBr': 20, 'NridgHt': 21, 'NoRidge': 22},
    'Condition1': {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'RRAn': 3, 'Rare': 4, 'PosN': 5},
    'Condition2': {'Rare': 0, 'Norm': 1},
    'BldgType': {'2fmCon': 0, 'Duplex': 1, 'Twnhs': 2, '1Fam': 3, 'TwnhsE': 4},
    'HouseStyle': {'SFoyer': 0, '1.5Fin': 1, 'Rare': 2, '1Story': 3, 'SLvl': 4, '2Story': 5},
    'RoofStyle': {'Gable': 0, 'Rare': 1, 'Hip': 2},
    'RoofMatl': {'CompShg': 0, 'Rare': 1},
    'Exterior1st': {'AsbShng': 0, 'Wd Sdng': 1, 'WdShing': 2, 'MetalSd': 3, 'Stucco': 4, 'Rare': 5, 'HdBoard': 6, 'Plywood': 7, 'BrkFace': 8, 'CemntBd': 9, 'VinylSd': 10},
    'Exterior2nd': {'AsbShng': 0, 'Wd Sdng': 1, 'MetalSd': 2, 'Wd Shng': 3, 'Stucco': 4, 'Rare': 5, 'HdBoard': 6, 'Plywood': 7, 'BrkFace': 8, 'CmentBd': 9, 'VinylSd': 10},
    'MasVnrType': {'Rare': 0, 'None': 1, 'BrkFace': 2, 'Stone': 3},
    'Foundation': {'Slab': 0, 'BrkTil': 1, 'CBlock': 2, 'Rare': 3, 'PConc': 4},
    'Heating': {'Rare': 0, 'GasW': 1, 'GasA': 2},
    'CentralAir': {'N': 0, 'Y': 1},
    'Electrical': {'Rare': 0, 'FuseF': 1, 'FuseA': 2, 'SBrkr': 3},
    'Functional': {'Rare': 0, 'Min2': 1, 'Mod': 2, 'Min1': 3, 'Typ': 4},
    'GarageType': {'Rare': 0, 'Detchd': 1, 'Basment': 2, 'Attchd': 3, 'BuiltIn': 4},
    'PavedDrive': {'N': 0, 'P': 1, 'Y': 2},
    'PoolQC': {'Missing': 0, 'Rare': 1},
    'MiscFeature': {'Rare': 0, 'Shed': 1, 'Missing': 2},
    'SaleType': {'COD': 0, 'Rare': 1, 'WD': 2, 'New': 3},
    'SaleCondition': {'Rare': 0, 'Abnorml': 1, 'Family': 2, 'Normal': 3, 'Partial': 4},
    'MSSubClass': {30: 0, 'Rare': 1, 190: 2, 90: 3, 160: 4, 50: 5, 85: 6, 70: 7, 80: 8, 20: 9, 75: 10, 120: 11, 60: 12}
}

In [194]:
for key, val in encoding_dict.items():
    test_data[key] = test_data[key].map(val)

In [195]:
[var for var in test_data.columns if test_data[var].isnull().sum() > 0]

[]

In [196]:
'asdf' in ['asdf']

True

In [197]:
# drop unexpected columns
na_columns = ['LotFrontage_na', 'MasVnrArea_na', 'GarageYrBlt_na']
for col in test_data.columns:
    if (col.endswith('_na')) and (not col in na_columns):
        test_data.drop(col, axis=1, inplace=True)

test_data.shape

(1459, 81)

### Feature Scaling

In [198]:
scaler = joblib.load('minmax_scaler.joblib')
scaled_data = scaler.transform(test_data)
test_data = pd.DataFrame(scaled_data, columns=test_data.columns)
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,0.750000,0.50,0.495064,0.0,1.0,1.0,0.000000,0.333333,1.0,0.0,...,0.0,0.75,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,0.0
1,0.750000,0.75,0.499662,0.0,1.0,1.0,0.333333,0.333333,1.0,0.5,...,0.0,0.00,0.0,1.0,0.454545,0.666667,0.75,0.0,0.0,0.0
2,1.000000,0.75,0.466207,0.0,1.0,1.0,0.333333,0.333333,1.0,0.0,...,0.0,0.75,1.0,0.0,0.181818,0.666667,0.75,0.0,0.0,0.0
3,1.000000,0.75,0.485693,0.0,1.0,1.0,0.333333,0.333333,1.0,0.0,...,0.0,0.00,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,0.0
4,0.916667,0.75,0.265271,0.0,1.0,1.0,0.333333,1.000000,1.0,0.0,...,0.0,0.00,1.0,0.0,0.000000,0.666667,0.75,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.333333,0.25,0.000000,0.0,1.0,1.0,0.000000,0.333333,1.0,0.0,...,0.0,0.00,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,1.0
1455,0.333333,0.25,0.000000,0.0,1.0,1.0,0.000000,0.333333,1.0,0.0,...,0.0,0.00,1.0,0.0,0.272727,0.666667,0.25,0.0,0.0,0.0
1456,0.750000,0.75,0.751625,0.0,1.0,1.0,0.000000,0.333333,1.0,0.0,...,0.0,0.00,1.0,0.0,0.727273,0.666667,0.25,0.0,0.0,0.0
1457,0.500000,0.75,0.400718,0.0,1.0,1.0,0.000000,0.333333,1.0,0.0,...,0.0,0.75,0.5,1.0,0.545455,0.666667,0.75,0.0,0.0,1.0


In [199]:
test_data.to_csv('scaled_test_data.csv')

### Feature Selection

In [217]:
selected_feats = pd.read_csv('selected_features.csv')
selected_feats = selected_feats.T.values.tolist()[0]

In [219]:
test_data = test_data[selected_feats]

### Import the Model

In [221]:
model = joblib.load('linear_regression.joblib')

### Make the new predictions

In [223]:
np.exp(model.predict(test_data))

array([113422.55297118, 154886.22617423, 176126.49165119, ...,
       155034.52741811, 119950.45101078, 209904.12437597])